### 使用 apyori

# ☆ Instacart Feast ☆

- <font size=3>https://reurl.cc/aro5bD</font>

In [ ]:
!pip3 install --upgrade mlxtend
!pip3 install apyori

# Importing Libraries

In [ ]:
try:
    from google.colab import drive, files
    in_colab = True
except ModuleNotFoundError:
    in_colab = False

if in_colab:
    home_dir = ''
    drive.mount('/content/drive')
    groot_dir = '/content/drive/My Drive/adventures/'
else:
    from pathlib import Path
    home_dir = str(Path.home())
    groot_dir = home_dir + '/Google Drive/adventures/'

import matplotlib as mpl
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=14)
mpl.rc('ytick', labelsize=14)
mpl.rc('font', size=14)

import datetime
from dateutil.relativedelta import *
import matplotlib.pyplot as plt
import sklearn
assert sklearn.__version__ >= "0.20"
import seaborn as sns
import pandas as pd
import numpy as np
import math
import os
import sys
import gdown
import requests
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
from pandas.plotting import register_matplotlib_converters

figure_dir = groot_dir + 'apriori/'
data_dir = groot_dir + 'apriori/'

gfigure = lambda name: figure_dir + name + '.png'
output_fig = lambda name: plt.savefig( gfigure(name), dpi = 300)

local_time = lambda x, offset: x + relativedelta(hours= offset)

def local_now(hours = 8):
    return datetime.datetime.now() + \
        relativedelta(hours = hours if in_colab else 0)

def print_now():
    return print(local_now())

def print_local_now():
    return print('Local Time:', local_now())

def DropboxLink(did, fname):
    return 'https://dl.dropboxusercontent.com/s/%s/%s' % \
    (did, fname)

def fetch_gdrive_file(fid, local_save):
    remote_url = 'https://drive.google.com/uc?id=' + fid
    gdown.download(remote_url, local_save, quiet = False)

def fetch_file_via_requests(url, save_in_dir):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    output_fpath = save_in_dir + local_filename
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(output_fpath, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    # f.flush()
    return output_fpath

def round_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.ceil(n * multiplier) / multiplier

def round_down(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n * multiplier) / multiplier
    
def start_plot(figsize=(10, 8), style = 'whitegrid'):
    fig = plt.figure(figsize=figsize)
    gs = fig.add_gridspec(1,1)
    plt.tight_layout()
    with sns.axes_style(style):
        ax = fig.add_subplot(gs[0,0])
    return ax
    
print('\nRunning on %s' % sys.platform)
print('Python Version', sys.version)
print('Data storage points to ==>', data_dir)

print('\nThis module is aimed to learn market basket analysis...') 
print('\nLibraries and dependenciess imported') 
print_now()


Running on darwin
Python Version 3.8.5 (v3.8.5:580fbb018f, Jul 20 2020, 12:11:27) 
[Clang 6.0 (clang-600.0.57)]
Data storage points to ==> /Users/roger/Google Drive/adventures/apriori/

This module is aimed to learn market basket analysis...

Libraries and dependenciess imported
2020-07-27 15:12:50.269174


# Warm Up：store_data.csv

- [Association Rule Mining via Apriori Algorithm in Python](https://stackabuse.com/association-rule-mining-via-apriori-algorithm-in-python/)
- [data on Github](https://github.com/gitganeshnethi/Datasets/blob/master/store_data.csv)
- [Complete guide to Association Rules](https://towardsdatascience.com/complete-guide-to-association-rules-2-2-c92072b56c84)

### 將資料載入記憶體





In [ ]:
fetch_file_via_requests(
    DropboxLink('9wcgqcl99efi12i', 'store_data.csv'), data_dir
)

'/Users/roger/Google Drive/adventures/apriori/store_data.csv'

In [ ]:
csv_path = os.path.join(data_dir, 'store_data.csv')
df = pd.read_csv(csv_path, header=None)
df.fillna('', inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,,,,,,,,,,,,,,,,,
2,chutney,,,,,,,,,,,,,,,,,,,
3,turkey,avocado,,,,,,,,,,,,,,,,,,
4,mineral water,milk,energy bar,whole wheat rice,green tea,,,,,,,,,,,,,,,


- odata : order data (list of list)
- item_list: list of goods

we need an array conatining all purachasing information. the array must be of the form 

(n_orders, n_items)



### apyori

- [PyPI apyori](https://pypi.org/project/apyori/)

#### apyori_output()

In [ ]:
#
# apyori output is unreadable, I tried to compile
# a dataframe which is more meaningful and readable
#
def apyori_output(apyori_results):
    cols = ['antecedents', 'consequents', 'support',
    'confidence', 'lift', 'itemsets' ]
    df = pd.DataFrame(columns = cols)

    for rule in apyori_results:
        new_row = {
                'antecedents': rule[2][0].items_base,
                'consequents': rule[2][0].items_add,
                'support': rule[1],
                'confidence':rule[2][0].confidence,
                'lift': rule[2][0].lift,
                'itemsets': rule[0]}
        df = df.append(new_row, ignore_index=True)
    return df

## 使用 mlxtend 套件

- [mlxtend frequent_patterns](http://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/)
- [mlxtend source repository](https://github.com/rasbt/mlxtend/blob/master/mlxtend/preprocessing/transactionencoder.py)

In [ ]:
import mlxtend
mlxtend.__version__

### mlxtend 0.14.2 不支援 fpgrowth




1. 
```
!pip3 install --upgrade mlxtend
```
2. Restart Runtime

### mlxtend 0.14.2 支援 apriori

# ★ Instacart Orders ★

- [Instacart](https://www.instacart.com/)
- [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)
- [The Instacart Online Grocery Shopping Dataset 2017 Data Descriptions](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b)
- [Instacart Market Basket Analysis](https://www.kaggle.com/c/instacart-market-basket-analysis/overview)
- [Association Rules Mining/Market Basket Analysis](https://www.kaggle.com/datatheque/association-rules-mining-market-basket-analysis)

### 下載檔案

In [ ]:
fetch_file_via_requests(
    DropboxLink('q5lrk9lucegzb8k', 'aisles.csv'), 
    data_dir )
fetch_file_via_requests(
    DropboxLink('20cdvqb7s50g5au', 'departments.csv'), 
    data_dir )
fetch_file_via_requests(
    DropboxLink('engd72pjwielb8p', 'orders.csv'), 
    data_dir )
fetch_file_via_requests(
    DropboxLink('ae0u82l9psj9ba4', 'products.csv'), 
    data_dir )
fetch_file_via_requests(
    DropboxLink('6qy0gksa22yqkyk', 'order_products__train.csv'), 
    data_dir )
fetch_file_via_requests(
    DropboxLink('vxlujw1xhqxt8sl', 'order_products__prior.csv'), 
    data_dir )

### 將資料載入記憶體

In [ ]:
%%time

# order_prior = \
#     pd.read_csv(os.path.join(data_dir, 'order_products__prior.csv'))

order_train = \
    pd.read_csv(os.path.join(data_dir, 'order_products__train.csv'))

aisles = \
     pd.read_csv(os.path.join(data_dir, 'aisles.csv'))

prods = \
    pd.read_csv(os.path.join(data_dir, 'products.csv'))

order_info = \
    pd.read_csv(os.path.join(data_dir, 'orders.csv'))

departments = \
    pd.read_csv(os.path.join(data_dir, 'departments.csv'))

print('Instacart data loaded and transformed into DataFrame(s)')
print_now()

Instacart data loaded and transformed into DataFrame(s)
2020-07-27 15:36:43.369891
CPU times: user 1.57 s, sys: 369 ms, total: 1.94 s
Wall time: 1.98 s


### merge transactions dataframe

In [ ]:
%%time
tx = pd.merge(prods, order_train, how = 'inner')

CPU times: user 245 ms, sys: 116 ms, total: 361 ms
Wall time: 381 ms


## preparing for order-lists

### 讀入已經處理好的 order_train pickle

In [ ]:
%%time
import pickle
from_disk_file = os.path.join(data_dir, 'order_train list.pkl')
with open(from_disk_file, 'rb') as f:
    order_data = pickle.load(f)
    f.close()
print('order_data loaded from pickled object')
print('%d bytes allocated for order_data\n' % sys.getsizeof(order_data))
print_now()

order_data loaded from pickled object
1080104 bytes allocated for order_data

2020-07-27 15:41:47.934040
CPU times: user 114 ms, sys: 11.6 ms, total: 126 ms
Wall time: 127 ms


## apyori


## mlxtend

- [mlxtend apriori](http://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/)

In [ ]:
%%time
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

te = TransactionEncoder()
te_ary = te.fit_transform(order_data)
orders_to_be_mined = pd.DataFrame(te_ary, columns=te.columns_)
print('MLXTEND: Transcation Encoder Completed')
print_now()
print('')

frequent_itemsets = apriori(orders_to_be_mined, 
        min_support=0.005, 
        use_colnames=True)
print('MLXTEND: frequent itemsets generated')
print_now()

MLXTEND: Transcation Encoder Completed
2020-07-22 14:02:29.099738

MLXTEND: frequent itemsets generated
2020-07-22 14:10:12.738484
CPU times: user 5min 10s, sys: 1min 48s, total: 6min 58s
Wall time: 7min 46s


In [ ]:
import pickle
import datetime

def today_str():
    a = datetime.date.today()
    return f'{a.month:02d}-{a.day:02d}'

dump_file = os.path.join(data_dir, 
    '(train) frequent itemset %s.pkl' % today_str())
f = open(dump_file, 'wb')
pickle.dump(frequent_itemsets, f)
f.close()

### generate rules

## 有幾筆資料？

## 消費者下單次數

## 每一單買幾個商品 basket_size

## best sellers 銷售排行榜

## 消費者偏好的購物時間 when do people buy

## 商品重複購買的比率

重購比例最高的十個商品：

- n: 每個產品出現的次數，也就是被購買的總數
- repeat_buying_orders: 若每個產品出現時， reordered 欄位也是1，表示這是重複購買，我們要計算這種訂單的總數
-----------

$產品重購比例=\frac{重複購買次數}{總購買數}$

## 放進購物車的順序與是否為重複購買的關聯


## 哪些商品第一個被擺入購物車？

let product_id = k

- 分母: k 被購買的總次數
- 分子 #1: k 是重購，而且是第一個被加入購物車的次數 .sum()
- 分子 #2: k 第一個被加入購物車的次數 .count()

## 顧客分群 Segmentation

# End of File